https://docs.fast.ai/tutorial.text#Fine-tuning-a-language-model-on-IMDb

In [1]:
import pickle

import pandas as pd
from sklearn.model_selection import train_test_split

from english_words import english_words_lower_set
import fastai
from fastai.text.all import *

In [2]:
df_quests = pd.read_csv("../data/quests/quest_objectives_entities.csv", sep=";")

In [3]:
df_quests.head()

,source,id,title,objectives,objectives_entities
0,skyrim,55074,A Blade in the Dark,Talk to Delphine \nLocate the dragon burial site \nKill the Dragon Sahloknir \nReturn to Delphine,Talk to xperx Locate the dragon burial site xorgx the xmiscx Sahloknir Return to xperx
1,skyrim,55212,A Chance Arrangement,Talk to Brynjolf about joining the Thieves Guild \nMeet Brynjolf during daytime \nSteal Madesi's ring from his strongbox \nPlant Madesi's ring \nSpeak to Brynjolf,Talk to xperx about joining the Thieves Guild Meet xperx during daytime Steal xorgx 's ring from his strongbox Plant xorgx 's ring Speak to xperx
2,skyrim,51114,A Cornered Rat,Talk to Brynjolf \nSearch the Ratway for Esbern's hideout \nFind Esbern in the Ratway Warrens \nTalk to Esbern,Talk to xperx Search the xlocx for xperx 's hideout xmiscx xperx in the xlocx Warrens Talk to xperx
3,skyrim,52017,A Daedra's Best Friend,Talk with Barbas \nTravel with Barbas to the shrine of Clavicus Vile \nJoin Barbas outside Haemar's Shame \nRetrieve the Rueful Axe with Barbas \nReturn to the shrine of Clavicus Vile with Barbas and the Rueful Axe \nGive the Rueful Axe to Clavicus Vile or kill Barbas with the Rueful Axe \nSpeak to Clavicus Vile,Talk with xperx Travel with xperx to the shrine xmiscx Clavicus Vile Join xperx outside Haemar 's Shame Retrieve the Rueful Axe with xperx Return to the shrine xmiscx Clavicus Vile with xperx xorgx the Rueful Axe Give the Rueful Axe to Clavicus Vile or kill xperx with the Rueful Axe Speak to Clavicus Vile
4,skyrim,63392,A False Front (Imperials),Find the Stormcloak courier \nRetrive the Stormcloak courier's package \nBring the documents to Legate Rikke \nBring the forged documents to Frorkmar Banner-Torn,xmiscx the xorgx courier Retrive the xorgx courier 's package Bring the documents to Legate xperx Bring the forged documents to Frorkmar Banner-Torn


In [4]:
df_quests["text"] = df_quests.apply(lambda x: (x["title"] + " > " + x["objectives_entities"].replace(".", " > ")).lower(), axis=1)

In [5]:
df_quests.sample(3)["text"]

277                                                                                                                                                                                                                                                                  the golden claw (skyrim) > retrieve the xlocx claw cut arvel down retrieve the xlocx claw xmiscx the secret xmiscx bleak falls xorgx bring the claw to xlocx
117                                                                                                                                                                                                                                                                                                      good intentions > speak with xorgx listen to xorgx follow xorgx xmiscx the augur xmiscx xlocx report back to xperx xorgx
455    the isle of mists (quest) > xperx to xorgx  >  talk to friends who could help defeat the xorgx xorgx ask them to join your fight  >  get on a boat  >  travel

In [6]:
with open("../data/raw/gameofthrones.txt") as infile:
    got_txt = infile.readlines()
got_txt = [l for l in got_txt if l != "\n"]

In [7]:
df_got = pd.DataFrame({"text": got_txt})

In [9]:
df = pd.concat([df_got["text"], df_quests["text"]]).to_frame()

In [10]:
lm_data = TextDataLoaders.from_df(df, text_col="text", is_lm=True, seed=7)

/home/tiago/anaconda3/envs/creative-ai/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [11]:
lm_data.show_batch(max_n=3)

,text,text_
0,"xxbos “ gods , xxmaj catelyn , xxmaj sansa is only eleven , ” xxmaj ned said . “ and xxmaj joffrey … xxmaj joffrey is … ” xxbos a column of riders emerged from beneath the portcullis with a clink of steel and a clatter of hooves . xxmaj clegane stepped close to the king , one hand on the hilt of his longsword . xxmaj the visitors were dinted and","“ gods , xxmaj catelyn , xxmaj sansa is only eleven , ” xxmaj ned said . “ and xxmaj joffrey … xxmaj joffrey is … ” xxbos a column of riders emerged from beneath the portcullis with a clink of steel and a clatter of hooves . xxmaj clegane stepped close to the king , one hand on the hilt of his longsword . xxmaj the visitors were dinted and haggard"
1,"battle of the xxmaj blackwater changed all . xxmaj the rose and the lion joined there , to shatter xxmaj stannis xxmaj baratheon ’s host and burn his fleet to ashes . ” xxbos xxmaj the bloodstained messenger shook his head . “ our outriders had been vanishing . xxmaj marq xxmaj piper ’s work , we thought . xxmaj the ones who did come back had seen nothing . ” xxbos","of the xxmaj blackwater changed all . xxmaj the rose and the lion joined there , to shatter xxmaj stannis xxmaj baratheon ’s host and burn his fleet to ashes . ” xxbos xxmaj the bloodstained messenger shook his head . “ our outriders had been vanishing . xxmaj marq xxmaj piper ’s work , we thought . xxmaj the ones who did come back had seen nothing . ” xxbos “"
2,". xxmaj and someone will need to wake xxmaj joffrey . ” xxbos xxmaj she took a deep breath to calm herself . “ i thought i was alone , xxup i … ” xxmaj she glanced away . xxbos “ one against seven ? ” xxmaj halder gave a signal . xxmaj the boys spread out , surrounding him . xxbos “ who do you think sends the wind , if","xxmaj and someone will need to wake xxmaj joffrey . ” xxbos xxmaj she took a deep breath to calm herself . “ i thought i was alone , xxup i … ” xxmaj she glanced away . xxbos “ one against seven ? ” xxmaj halder gave a signal . xxmaj the boys spread out , surrounding him . xxbos “ who do you think sends the wind , if not"


In [12]:
learn = language_model_learner(lm_data, AWD_LSTM, metrics=[accuracy, Perplexity()], path="../models/")

In [13]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.195041,3.725751,0.285722,41.502407,00:58


In [14]:
learn.save('1epoch')

Path('../models/models/1epoch.pth')

In [15]:
learn = learn.load('1epoch')

In [16]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.924222,3.620647,0.300250,37.361721,01:08
1,3.786116,3.528270,0.311006,34.064983,01:09
2,3.673791,3.448728,0.320587,31.460360,01:08
3,3.563691,3.394011,0.327964,29.785168,01:08
4,3.492585,3.358512,0.331786,28.746376,01:08
5,3.406917,3.333518,0.334803,28.036810,01:08
6,3.356275,3.318056,0.337136,27.606640,01:08
7,3.307809,3.309276,0.338147,27.365303,01:08
8,3.283530,3.306637,0.338591,27.293173,01:08
9,3.268065,3.306350,0.338578,27.285339,01:12


In [17]:
learn.save('finetuned_book')

Path('../models/models/finetuned_book.pth')

In [19]:
with open("../models/models/book_vocab.pkl", "wb") as outfile:
    pickle.dump(lm_data.vocab, outfile)

---

In [21]:
df_quests.head()["title"]

0          A Blade in the Dark
1         A Chance Arrangement
2               A Cornered Rat
3       A Daedra's Best Friend
4    A False Front (Imperials)
Name: title, dtype: object

In [27]:
quest_title = "Swords in the dark"

n_goals = 5
[learn.predict(quest_title.lower() + " >", n_words=15) for _ in range(n_goals)]

['swords in the dark > read the letter you found > xmiscx the agent near the kitchen in the xorgx',
 "swords in the dark > get xperx 's payment on the xmiscx giver give the rope to xperx",
 'swords in the dark > talk to xperx > talk to the woman > talk to the xperx xorgx xorgx',
 'swords in the dark > xmiscx the target where the doorway is using your witcher senses > fist the gueren',
 'swords in the dark > pass the cried raven xmiscx broken eagle jacks against the crow xmiscx the star xmiscx']

In [44]:
quest_title = "Swords in the dark > enter shrine"

n_goals = 5
[learn_quests.predict(quest_title.lower() + " >", n_words=15) for _ in range(n_goals)]

["swords in the dark > enter shrine > execute a spell on bearers in ruby thugs ' n retrieve your escort hut '",
 'swords in the dark > enter shrine > talk to the vampire drummers if whickered is done with monsters : wolf - boots',
 "swords in the dark > enter shrine > captain 's secret : bobbed > read the letter you found > clanged the",
 'swords in the dark > enter shrine > talk to scowled get defeat activate the home > befriend the stranger > search the',
 'swords in the dark > enter shrine > learn elderly iron sword bring probing some high - ankle rings caps with arms come']

---

In [28]:
lm_data_quests = TextDataLoaders.from_df(df_quests, text_col="text", is_lm=True, seed=7, text_vocab=lm_data.vocab)

/home/tiago/anaconda3/envs/creative-ai/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [30]:
lm_data_quests.show_batch(max_n=3)

,text,text_
0,xxbos ciri 's story : the race > beat the baron in the horse race > xxbos armed assault > knock on the door to check on the xxunk xmiscx the house > xxbos blood 's honor > speak to xperx xxunk collect a xlocx xmiscx 's head wipe out the xlocx xxunk ( optional ) return to xperx xxbos xxunk xxunk > xmiscx the scribe in the xorgx xorgx sign up,ciri 's story : the race > beat the baron in the horse race > xxbos armed assault > knock on the door to check on the xxunk xmiscx the house > xxbos blood 's honor > speak to xperx xxunk collect a xlocx xmiscx 's head wipe out the xlocx xxunk ( optional ) return to xperx xxbos xxunk xxunk > xmiscx the scribe in the xorgx xorgx sign up for
1,"xperx > meet xperx at the seven cats inn > ( 80 ) xperx to xlocx xorgx look for xperx > ask xperx what he knows about xperx > xmiscx xperx > xmiscx the shrine where xperx xxunk > xorgx xperx > ( ? ) search xperx 's body > meet xperx at the nowhere inn > meet xperx on xxunk square , near xperx 's residence > ( xxunk ) decide","> meet xperx at the seven cats inn > ( 80 ) xperx to xlocx xorgx look for xperx > ask xperx what he knows about xperx > xmiscx xperx > xmiscx the shrine where xperx xxunk > xorgx xperx > ( ? ) search xperx 's body > meet xperx at the nowhere inn > meet xperx on xxunk square , near xperx 's residence > ( xxunk ) decide xxunk"
2,you dare ! > xmiscx out more about the contract > xmiscx out what 's stealing the old woman 's chickens using your witcher senses > convince the old woman to take in the children > xxbos shahvee 's amulet > xmiscx the amulet xmiscx xxunk for shahvee return the amulet xmiscx xxunk to shahvee xxbos xxunk ( skyrim ) > help xperx reach the top xmiscx xxunk xlocx defeat xperx 's,dare ! > xmiscx out more about the contract > xmiscx out what 's stealing the old woman 's chickens using your witcher senses > convince the old woman to take in the children > xxbos shahvee 's amulet > xmiscx the amulet xmiscx xxunk for shahvee return the amulet xmiscx xxunk to shahvee xxbos xxunk ( skyrim ) > help xperx reach the top xmiscx xxunk xlocx defeat xperx 's mother


In [32]:
learn_quests = language_model_learner(lm_data_quests, AWD_LSTM, metrics=[accuracy, Perplexity()], path="../models/")

In [33]:
learn_quests.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,7.785580,6.885910,0.146267,978.391602,00:01


In [34]:
learn_quests.unfreeze()
learn_quests.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,7.034710,6.015705,0.202148,409.814697,00:01
1,6.606172,5.423470,0.223850,226.664398,00:01
2,6.235723,5.008296,0.290690,149.649521,00:01
3,5.940209,4.652508,0.329536,104.847588,00:01
4,5.688892,4.444615,0.321181,85.167114,00:01
5,5.477461,4.250602,0.343750,70.147614,00:01
6,5.297235,4.183322,0.348633,65.583359,00:01
7,5.142070,4.058821,0.357856,57.906010,00:01
8,5.016280,4.043697,0.351671,57.036839,00:01
9,4.913169,4.044274,0.351888,57.069729,00:01


In [35]:
learn_quests.save('finetuned_quests')

Path('../models/models/finetuned_quests.pth')

In [46]:
quest_title = "Swords in the dark"

n_goals = 5
[learn_quests.predict(quest_title.lower() + " >", n_words=15) for _ in range(n_goals)]

['swords in the dark > speak to > read information you found > quest the key to the collect black',
 'swords in the dark > read your shouts bring the void to the injured sodden > follow the … wonder',
 'swords in the dark > > speak to pick up > get a shipment from the former continue to the',
 'swords in the dark > speak to stubs ( weddings ) say what happened to the murderer : question what',
 'swords in the dark > read the letter you found > minds > optional in the boar skull >']

In [43]:
quest_title = "Swords in the dark > enter shrine"

n_goals = 5
[learn_quests.predict(quest_title.lower() + " >", n_words=15) for _ in range(n_goals)]

['swords in the dark > enter shrine > hunt there > get the reward > the responsibility for carpets > collect seeds',
 'swords in the dark > enter shrine > find the elder assassin > seek the puzzle beans extra dice > friends with courtesies',
 "swords in the dark > enter shrine > talk to m > bar 's curse > retrieve fifty collect hand - encrusted",
 'swords in the dark > enter shrine > grace for the traitors > read the letter you found near the camp >',
 "swords in the dark > enter shrine > check for dracarys ' secrets : > thieves in the cave > 0 / 2"]

In [45]:
quest_title = "Swords in the dark > enter shrine > hunt there"

n_goals = 5
[learn_quests.predict(quest_title.lower() + " >", n_words=15) for _ in range(n_goals)]

["swords in the dark > enter shrine > hunt there > take the sword dragon 's sword > solve in the blood > talk to",
 'swords in the dark > enter shrine > hunt there > put on the steel sword upgrade sword > you can unlocked lawless > talk to',
 'swords in the dark > enter shrine > hunt there > meet massive grave > askew around mummers manacles > talk to mortified fleshless > speak',
 'swords in the dark > enter shrine > hunt there > begin talks with the seamen in survivors discover that stiffening ( if let speak )',
 'swords in the dark > enter shrine > hunt there > deal with the thieves legend > search the woods by floor > complete the final']

---

# backup

In [55]:
learn.fine_tune(10, 1e-2)

epoch,train_loss,valid_loss,time
0,6.589756,5.739772,00:00


epoch,train_loss,valid_loss,time
0,5.706264,4.869517,00:01
1,5.222791,4.101242,00:01
2,4.813826,3.628067,00:01
3,4.486436,3.427560,00:01
4,4.238911,3.203441,00:01
5,4.023324,3.121645,00:01
6,3.852403,3.077629,00:01
7,3.714095,3.050021,00:01
8,3.595885,3.038499,00:01
9,3.501235,3.035754,00:01


In [57]:
learn.predict(title.lower() + ' > ', n_words=20)

'a blade in the dark > talk to the black man > collect boots word from killing the baron of his home . > retrieve gold'

In [58]:
learn.predict(title.lower() + ' > ', n_words=20)

'a blade in the dark > bring power to lower giver the family battle > look for the guardian leader fort in sirens . >'

In [59]:
title = "The Crimson Conspiracy"

In [60]:
learn.predict(title.lower() + ' > ', n_words=20)

'the crimson xxunk > locate high and water vilkas notes to the 25 citizens of the ruins of woe against earth . >'

In [61]:
title = "A curious looking egg"

In [62]:
learn.predict(title.lower() + ' > ', n_words=20)

'a xxunk xxunk xxunk > find the ring on orchard of wood > find the way to the obelisk . > use the obelisk'